In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import SingleIdTokenIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = SingleIdTokenIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:01, 139.37it/s]
251it [00:00, 266.95it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x1a24c8c080>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

In [17]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

02/07/2019 17:53:35 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.
100%|██████████| 267/267 [00:00<00:00, 14279.62it/s]


### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[2952,  166,   34,  ...,    0,    0,    0],
          [  42,  613,  986,  ...,    0,    0,    0],
          [ 385,   94,   95,  ...,    0,    0,    0],
          ...,
          [   5,  498,   45,  ...,    4,  749,    3],
          [   5,  672,  107,  ...,    6,  374,    8],
          [ 262,   46,    4,  ..., 1210,    0,    0]])},
 'id': ['008344a80c43b8c9',
  '005306a4c109dab3',
  '008f1b06428778fe',
  '002264ea4d5f2887',
  '0038f191ffc93d75',
  '006d11791d76b9f3',
  '001d8e7be417776a',
  '004176f28a17bf45',
  '00a981826dc7c952',
  '005e6b1369cbe377',
  '00a317acddff8a62',
  '006a4cdda4960588',
  '002746baedcdff10',
  '007e1e47cd0e2fec',
  '00151a9f93c6b059',
  '001cadfd324f8087',
  '000cfee90f50d471',
  '0016e01b742b8da3',
  '0006f16e4e9f292e',
  '001ee16c46a99262',
  '003b9f448ee4a29d',
  '0060062dd4db5195',
  '0048de0c9422f64f',
  '001d874a4d3e8813',
  '004b103182fb1eab',
  '00822d0d01752c7e',
  '00548f16a392d8ed',
  '009565ee1bc64e68',
  '001810bf8c45b

In [23]:
batch["tokens"]["tokens"]

tensor([[2952,  166,   34,  ...,    0,    0,    0],
        [  42,  613,  986,  ...,    0,    0,    0],
        [ 385,   94,   95,  ...,    0,    0,    0],
        ...,
        [   5,  498,   45,  ...,    4,  749,    3],
        [   5,  672,  107,  ...,    6,  374,    8],
        [ 262,   46,    4,  ..., 1210,    0,    0]])

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 100])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

token_embedding = Embedding(num_embeddings=config.max_vocab_size + 2,
                            embedding_dim=300, padding_index=0)
# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

In [28]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(),
                                                        config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[2952,  166,   34,  ...,    0,    0,    0],
         [  42,  613,  986,  ...,    0,    0,    0],
         [ 385,   94,   95,  ...,    0,    0,    0],
         ...,
         [   5,  498,   45,  ...,    4,  749,    3],
         [   5,  672,  107,  ...,    6,  374,    8],
         [ 262,   46,    4,  ..., 1210,    0,    0]])}

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 0, 0]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[-0.0383,  0.0800, -0.0317,  0.0283,  0.0172, -0.0236],
        [-0.0398,  0.0806, -0.0331,  0.0289,  0.0170, -0.0243],
        [-0.0387,  0.0807, -0.0297,  0.0295,  0.0170, -0.0230],
        [-0.0386,  0.0815, -0.0327,  0.0297,  0.0152, -0.0263],
        [-0.0367,  0.0808, -0.0306,  0.0292,  0.0149, -0.0246],
        [-0.0390,  0.0815, -0.0305,  0.0276,  0.0160, -0.0249],
        [-0.0393,  0.0814, -0.0302,  0.0274,  0.0173, -0.0252],
        [-0.0396,  0.0805, -0.0303,  0.0269,  0.0167, -0.0240],
        [-0.0379,  0.0795, -0.0300,  0.0281,  0.0144, -0.0260],
        [-0.0385,  0.0804, -0.0310,  0.0263,  0.0171, -0.0263],
        [-0.0387,  0.0807, -0.0317,  0.0268,  0.0157, -0.0245],
        [-0.0381,  0.0792, -0.0317,  0.0292,  0.0151, -0.0237],
        [-0.0371,  0.0806, -0.0309,  0.0270,  0.0156, -0.0257],
        [-0.0389,  0.0818, -0.0308,  0.0271,  0.0154, -0.0245],
        [-0.0372,  0.0807, -0.0321,  0.0259,  0.0154, -0.0235],
        [-0.0363,  0.0805, -0.0310,  0.0

In [36]:
model(**batch)

{'class_logits': tensor([[-0.0383,  0.0800, -0.0317,  0.0283,  0.0172, -0.0236],
         [-0.0398,  0.0806, -0.0331,  0.0289,  0.0170, -0.0243],
         [-0.0387,  0.0807, -0.0297,  0.0295,  0.0170, -0.0230],
         [-0.0386,  0.0815, -0.0327,  0.0297,  0.0152, -0.0263],
         [-0.0367,  0.0808, -0.0306,  0.0292,  0.0149, -0.0246],
         [-0.0390,  0.0815, -0.0305,  0.0276,  0.0160, -0.0249],
         [-0.0393,  0.0814, -0.0302,  0.0274,  0.0173, -0.0252],
         [-0.0396,  0.0805, -0.0303,  0.0269,  0.0167, -0.0240],
         [-0.0379,  0.0795, -0.0300,  0.0281,  0.0144, -0.0260],
         [-0.0385,  0.0804, -0.0310,  0.0263,  0.0171, -0.0263],
         [-0.0387,  0.0807, -0.0317,  0.0268,  0.0157, -0.0245],
         [-0.0381,  0.0792, -0.0317,  0.0292,  0.0151, -0.0237],
         [-0.0371,  0.0806, -0.0309,  0.0270,  0.0156, -0.0257],
         [-0.0389,  0.0818, -0.0308,  0.0271,  0.0154, -0.0245],
         [-0.0372,  0.0807, -0.0321,  0.0259,  0.0154, -0.0235],
         

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.6964, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

# Train

In [40]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [41]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [42]:
metrics = trainer.train()

02/07/2019 17:53:40 - INFO - allennlp.training.trainer -   Beginning training.
02/07/2019 17:53:40 - INFO - allennlp.training.trainer -   Epoch 0/1
02/07/2019 17:53:40 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 724.332544
02/07/2019 17:53:40 - INFO - allennlp.training.trainer -   Training
loss: 0.6926 ||: 100%|██████████| 5/5 [00:04<00:00,  1.22it/s]
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -                     Training |  Validation
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -   loss          |     0.693  |       N/A
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -   cpu_memory_MB |   724.333  |       N/A
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:04
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:04
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -   Epoch 1/1
02/07/2019 17:53:45 - INFO - allennlp.training.trainer -   Peak CPU memory

# Generating Predictions

AllenNLP is slightly lacking in its ability to convert datasets to predictions (though it has extensive support for converting single examples to predictions). Therefore, we'll write our own Predictor class to handle this job for us.

Thankfully, a lot of the tools we used eariler can easily be extended to prediction. Here's how.

In [43]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

We'll need a iterator that goes sequentially over our data. We'll use the 

In [44]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [45]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 

100%|██████████| 4/4 [00:00<00:00, 14.44it/s]


# A Final Note on Predictors

AllenNLP also provides predictors that take strings as input and outputs model predictions. They're handy if you want to create simple demo or need to make predictions on entirely new data, but since we've already read data as datasets and want to preserve their order, we didn't use them above. 

In [46]:
from allennlp.predictors.sentence_tagger import SentenceTaggerPredictor

In [47]:
tagger = SentenceTaggerPredictor(model, reader)

In [48]:
tagger.predict("this tutorial was great!")

{'class_logits': [-0.07560957223176956,
  0.03300512582063675,
  -0.06693253666162491,
  -0.001402825117111206,
  -0.029230568557977676,
  -0.06722232699394226],
 'loss': 0.6762115359306335}

In [49]:
tagger.predict("this tutorial was horrible!")

{'class_logits': [-0.07560957223176956,
  0.03300512582063675,
  -0.06693253666162491,
  -0.001402825117111206,
  -0.029230568557977676,
  -0.06722232699394226],
 'loss': 0.6762115359306335}